In [2]:

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# input csv
ALL_DATA_csvpath = "../output_files/MegaDf.csv"

#import pretty print
import pprint as pprint

#for sleep
import time

In [3]:
ALL_DATA_csv = pd.read_csv(ALL_DATA_csvpath,index_col=0)

In [11]:
# Set occupation here
ALL_DATA_df_collist_no_occ = [col for col in ALL_DATA_csv.columns.tolist() if "occupations" not in col]
ALL_DATA_df_occ_collist = [col for col in ALL_DATA_csv.columns.tolist() if "occupations" in col\
                             if "RANK" not in col]
global careerList
careerList={}
for key in range(1,len(ALL_DATA_df_occ_collist)+1):
    careerList[str(key)] = ALL_DATA_df_occ_collist[key-1]
     
    
def jbReq():
    for i in list(careerList.keys()):
        print(f"{i}: {careerList[i]}")
    global OCC_CHOICE
    OCC_CHOICE = input("Which career field are you looking for information on?  Please input the career ID #: ")
    try:
        careerList[OCC_CHOICE]
        print("-----------------------------------------------------------------------\n")
        print(f"Valid choice. Good luck in your new venture in {careerList[OCC_CHOICE]}")
    except:
        print("-----------------------------------------------------------------------\n")
        print("No career found with that ID # please reference the list and try again.")
        print("-----------------------------------------------------------------------\n")
        jbReq()
#################################################### 
jbReq() #Actually run the function now
ALL_DATA_df_collist_oneocc = ALL_DATA_df_collist_no_occ + [careerList[OCC_CHOICE]] + [careerList[OCC_CHOICE]+"_RANK"]

1: Architecture and engineering occupations
2: Arts, design, entertainment, sports, and media occupations
3: Building and grounds cleaning and maintenance occupations
4: Business and financial operations occupations
5: Community and social service occupations
6: Computer and mathematical occupations
7: Construction and extraction occupations
8: Educational instruction and library occupations
9: Food preparation and serving related occupations
10: Healthcare practitioners and technical occupations
11: Healthcare support occupations
12: Installation, maintenance, and repair occupations
13: Legal occupations
14: Life, physical, and social science occupations
15: Management occupations
16: Office and administrative support occupations
17: Personal care and service occupations
18: Production occupations
19: Protective service occupations
20: Sales and related occupations
21: Transportation and material moving occupations
Which career field are you looking for information on?  Please input t

In [6]:
# Limit the dataframe's occupation sections based on user choice.
USER_CHOICE_df = ALL_DATA_csv[ALL_DATA_df_collist_oneocc].reset_index(drop=True)

# Add the ranked column for the occupation to the overall score, and re-rank the overall score for overall rank.
USER_CHOICE_df["Overall_SCORE"] = USER_CHOICE_df["overall_SCORE"] + USER_CHOICE_df[careerList[OCC_CHOICE] +"_RANK"]
USER_CHOICE_df["Overall_RANK"] = USER_CHOICE_df["Overall_SCORE"].rank()

In [7]:
USER_CHOICE_df["OCC CHOICE"] = careerList[OCC_CHOICE]
USER_CHOICE_df["OCC VALUE"] = USER_CHOICE_df[careerList[OCC_CHOICE]]
USER_CHOICE_df["OCC VALUE R"]=USER_CHOICE_df[careerList[OCC_CHOICE] +"_RANK"]

In [8]:
# NOTE: Do not change any of the code in this cell
# {OCC CHOICE}{OCC VALUE}
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>City (Overall Rank)</dt><dd>{City Name},{stateabbrev} ({overall_RANK})</dd>
<dt>City Population (Rank)</dt><dd>{city_pop2019} ({city_pop2019_RANK})</dd>
<dt>Expected Hourly Wage for {OCC CHOICE} (Rank)</dt><dd>{OCC VALUE} ({OCC VALUE R})</dd>
<dt>% Change in Businesses Active since Jan 19 (Rank)</dt><dd>{merchants_all} ({merchants_all_RANK})</dd>
<dt>New Case Rate - Covid (Rank)</dt><dd>{new_case_rate} ({new_case_rate_RANK})</dd>
<dt>% Change in Job Postings since Jan 19 (Rank)</dt><dd>{bg_posts} ({bg_posts_RANK})</dd>
<dt>Avg. Cost of Living (Rank)</dt><dd>{Average Cost of Living } ({Average Cost of Living _RANK})</dd>
<dt>Avg. Nearby Park Rating (Rank)</dt><dd>{Park Score} ({Park Score_RANK})</dd>
<dt>Avg. Nearby Hospital Rating (Rank)</dt><dd>{Health score} ({Health score_RANK})</dd>

</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
ALL_DATA_info = [info_box_template.format(**row) for index, row in USER_CHOICE_df.iterrows()]
locations = USER_CHOICE_df[["lat", "lon"]]

In [9]:


# Import API key
#from api_keys import g_key

import sys
path = "/Users/matth/Desktop"
sys.path.insert(0, path)
from API_Keys import api_key
g_key= api_key["Google Maps Platform"]

# configure gmaps with api key
gmaps.configure(api_key=g_key)

fig = gmaps.figure()

#HEAT LAYER
heat_layer = gmaps.heatmap_layer(locations, weights=USER_CHOICE_df["overall_RANK"], 
                                 dissipating=False, max_intensity=USER_CHOICE_df["overall_RANK"].max(),
                                 point_radius=1)

# Add marker layer
marker_layer = gmaps.marker_layer(locations, \
                                 info_box_content=[f"{info}" for info in ALL_DATA_info])

# Add layer and Display figure
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


fig
USER_CHOICE_df.sort_values(by = "OCC VALUE R")

ModuleNotFoundError: No module named 'API_Keys'